# Silver Layer Transformation - Lap Times

## Objective
Transform the `bronze.lap_times` table into a clean and structured Silver table.

## Steps
1. **Read** data from Bronze table `bronze.lap_times`.
2. **Select & Rename** key columns:
   - Driver info: `driver`, `driver_number`
   - Lap details: `lap_number`, `lap_time`, `position`, `track_status`
   - Sector times: `sector1_time`, `sector2_time`, `sector3_time`
   - Tyre & stint: `compound`, `stint`, `tyre_life`, `fresh_tyre`
   - Timing: `time`, `lap_start_time`, `lap_start_date`
   - Flags: `is_accurate`, `is_personal_best`, `deleted`, `deleted_reason`
3. **Data Cleaning**
   - Convert lap/sector times into milliseconds (`double` → `long`).
   - Standardize boolean flags (`is_accurate`, `is_personal_best`, `deleted`).
   - Cast numeric fields (`driver_number`, `lap_number`, `position`, `stint`, `tyre_life`).
4. **Write** results into the Silver schema as `silver.lap_times`.

## Output
- Clean and structured Silver table:
  `silver.lap_times`


In [0]:
from pyspark.sql import functions as F

# Read from Bronze
lap_times_bronze = spark.table("bronze.lap_times")

# Transformations
lap_times_silver = (
    lap_times_bronze
    .select(
        F.col("driver").alias("driver"),
        F.col("driver_number").cast("int").alias("driver_number"),
        F.col("lap_number").cast("int").alias("lap_number"),
        F.col("lap_time").cast("double").alias("lap_time_ms"),  # already in ms
        F.col("position").cast("int").alias("position"),
        F.col("track_status").alias("track_status"),

        # Sector times
        F.col("sector1_time").cast("double").alias("sector1_time_ms"),
        F.col("sector2_time").cast("double").alias("sector2_time_ms"),
        F.col("sector3_time").cast("double").alias("sector3_time_ms"),

        # Tyre & stint info
        F.col("compound").alias("compound"),
        F.col("stint").cast("int").alias("stint"),
        F.col("tyre_life").cast("int").alias("tyre_life"),
        F.col("fresh_tyre").cast("boolean").alias("fresh_tyre"),

        # Timing
        F.col("time").alias("lap_time_str"),
        F.col("lap_start_time").alias("lap_start_time"),
        F.col("lap_start_date").alias("lap_start_date"),

        # Flags
        F.col("is_accurate").cast("boolean").alias("is_accurate"),
        F.col("is_personal_best").cast("boolean").alias("is_personal_best"),
        F.col("deleted").cast("boolean").alias("deleted"),
        F.col("deleted_reason").alias("deleted_reason")
    )
    .dropDuplicates()
)

# Write to Silver
(
    lap_times_silver
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("silver.lap_times")
)
